In [70]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from copy import deepcopy
from torch.nn.utils.rnn import pad_sequence
from utils.reparam_module import ReparamModule
import numpy as np

In [74]:
dataset_name = 'yelp-small'
full_dataset_name = 'yelp-small'
min_frequency_dict = {
    'toys': 2,
    'sport': 2,
    'beauty': 3,
    'yelp-small': 2,
}
min_frequency = min_frequency_dict[dataset_name]
data = torch.load(f'./pattern-{dataset_name}-noise-50.pth')
print(len(data))

30431


In [72]:
# Example to show how to find frequent sequential patterns
# from a given sequence database subject to constraints
from sequential.seq2pat import Seq2Pat, Attribute

# Seq2Pat over 3 sequences
seq2pat = Seq2Pat(sequences=data, n_jobs=2, max_span=10)
# Patterns that occur at least twice (A-D)
patterns = seq2pat.get_patterns(min_frequency=min_frequency)
patterns_value = [_[:-1] for _ in patterns]
patterns_count = [_[-1] for _ in patterns]
print(len(patterns))

68819


In [75]:
original_train = torch.load(f'./dataset/{full_dataset_name}-noise-50/{dataset_name}/train_ori.pth')
len(original_train)

30431

In [76]:
seq_list_ori = [_[1][:_[3]] + [_[2][_[3] - 1]] for _ in original_train]

In [77]:
def is_sublist(sublst, lst):
    for element in sublst:
        try:
            ind = lst.index(element)
        except ValueError:
            return False
        lst = lst[ind+1:]
    return True

In [79]:
data_generation_pair = []
total_cnt = 0
from tqdm import tqdm
from random import shuffle
for seq_ori in tqdm(seq_list_ori):
    shuffle(patterns_value)
    cnt = 0
    for pattern in patterns_value:
        if is_sublist(pattern, seq_ori):
            data_generation_pair.append([seq_ori, pattern])
            cnt += 1
print(len(data_generation_pair))

100%|██████████| 30431/30431 [37:36<00:00, 13.48it/s] 

150941


In [ ]:
torch.save(data_generation_pair, f'./{dataset_name}-pair.pth')

In [65]:
logits = torch.randn(256, 5)
logits

tensor([[ 0.3337,  1.4089, -0.5874,  1.4942, -1.3465],
        [ 0.7478, -0.6908, -0.3150, -2.1382, -0.2867],
        [-1.2294, -0.4578,  0.8121,  2.3734,  0.0047],
        ...,
        [-1.4943, -1.0644,  0.3543,  1.9312, -0.0991],
        [ 0.8144,  0.3107,  1.4501, -0.6948,  1.1553],
        [ 1.3487,  0.7794,  0.8656, -0.1423,  0.8273]])

In [66]:
selection = F.gumbel_softmax(logits, dim=-1, tau=10)
selection

tensor([[0.1944, 0.1653, 0.1502, 0.1933, 0.2968],
        [0.1977, 0.2563, 0.2053, 0.1463, 0.1945],
        [0.2013, 0.1454, 0.2836, 0.1953, 0.1744],
        ...,
        [0.2263, 0.1605, 0.1759, 0.2303, 0.2070],
        [0.1963, 0.2086, 0.2197, 0.1821, 0.1934],
        [0.2307, 0.1758, 0.1713, 0.1953, 0.2269]])

In [69]:
loss = ( selection * torch.log(selection)).sum(-1).mean()
loss

tensor(-1.5988)